Login to Chameleon and download openrc.sh file from [here](https://testbed.expeca.proj.kth.se/project/api_access/openrc/). Upload it here next to this notebook and continue.

In the next cell, we setup the authentication method to be able to use Openstack clients.

In [ ]:
import os, re
from getpass import getpass

with open('nils-project-openrc.sh', 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

# password read from file

Install required packages and dependencies. Ignore the warnings.

In [ ]:
!pip uninstall -q -y moviepy
!pip install -q jedi
!pip install -q git+https://github.com/KTH-EXPECA/python-chi

Import packages

In [ ]:
import json
from loguru import logger
import chi.network, chi.container
from chi.expeca import reserve, list_reservations, unreserve_byid, get_container_status, wait_until_container_removed, get_available_publicips, get_worker_interfaces

Reserve the required equipment and resources

In [ ]:
experiment_duration = {"days": 1, "hours": 0}

# List of required leases with details
required_leases = [
    { 
        "type": "network",
        "name": "ep5g",
        "net_name": "ep5g-vip",
        "segment_id": "100",
        "duration": experiment_duration
    },
    {
        "type": "network",
        "name": "adv-04",
        "net_name": "adv-04",
        "segment_id": "134",
        "duration": experiment_duration,
    },
    {
        "type": "device",
        "name": "worker-08",
        "duration": experiment_duration,
    },
    {
        "type": "device",
        "name": "worker-03",
        "duration": experiment_duration,
    },
]

# List of previously existing leases
existing_leases = list_reservations(brief=True)

# Reserve outstanding resources
for required_lease in required_leases:
    lease_name_with_suffix = required_lease["name"] + "-lease"
    is_already_leased = any(
        existing_lease["name"] == lease_name_with_suffix 
        and existing_lease["status"] == "ACTIVE" 
        for existing_lease in existing_leases
    )
    if is_already_leased:
        logger.info(f"Resource {required_lease['name']} is already leased.")
        continue
    else:
        reserve(required_lease)

def get_reservation_id_by_name(name):
    for lease in list_reservations(brief=True):
        if name in lease['name']:
            return lease['reservation_id']

Project settings

In [ ]:
# User-defined edge network
edge_cidr = "10.70.70.0/24"
edge_gw_addr = "10.70.70.1"
edge_h1_addr = "10.70.70.10"
edge_h2_addr = "10.70.70.20"
edge_h3_addr = "10.70.70.30"
edge_h4_addr = "10.70.70.40"

# Fixed IP addresses
ue_gw_addr = "10.42.3.1"
ue_priv_addr = "10.42.3.2"
ep5g_gw_addr = "10.30.111.10"
public_gw_addr = "130.237.11.97"
ue_nat_cidr = "172.16.0.0/16"
ue1_nat_addr = "172.16.0.96"   # adv-04
ue2_nat_addr = "172.16.0.64"   # adv-05

# Routes
route_towards_ue = "-".join([ue_nat_cidr,edge_gw_addr])
route_towards_edge = "-".join([edge_cidr,ue_gw_addr])

# User-defined L2TP addresses
l2tp_cidr = "172.18.0.0/24"
l2tp_h1_addr = "172.18.0.2"
l2tp_h2_addr = "172.18.0.3"

# Assigning workers
discovery_worker ="worker-03"
edge_node_worker = "worker-03"
ue1_node_worker = "worker-08"
ue2_node_worker = "worker-08"

Create networks

In [ ]:
# Create edge-net for talker
try: 
    edgenet = chi.network.get_network("edge-net")
    edge_subnet = chi.network.get_subnet("edge-subnet")
    logger.info("edge-net already exists.")
except:
    edgenet = chi.network.create_network("edge-net")
    edge_subnet = chi.network.create_subnet(
        subnet_name = "edge-subnet", 
        network_id = edgenet["id"], 
        cidr = edge_cidr, 
        gateway_ip = edge_gw_addr, 
        enable_dhcp = False
    )
    logger.success("edge-net is created.")

Create routers

In [ ]:
ep5g_net = chi.network.get_network("ep5g-vip-net")

try:
    chi.network.get_router("edge-router")
    logger.info("Router already exists.")
except:
    router = chi.network.create_router("edge-router", "public")
    chi.network.add_subnet_to_router(router["id"], edge_subnet["id"])
    chi.network.add_subnet_to_router(router["id"], ep5g_net["subnets"][0])
    chi.network.add_route_to_router(router["id"],"172.16.0.0/16","10.30.111.10")
    logger.success("Router created and subnets added.")

Start discovery-server on edge ("host 1")

In [ ]:
available_ips = get_available_publicips()
discovery_addr_public = available_ips[-1]
logger.info(f"Public IP address will be {discovery_addr_public} for this container.")

discovery_container_name = "discovery-server"
discovery_image_name = "nilsjor/ros-humble-turtlebot:discovery-v0.7"

discovery_addr = edge_h1_addr
discovery_addr_l2tp = l2tp_h1_addr
tunnel_dest_addr = ue1_nat_addr

discovery_env_vars = {
    "ROS_DISCOVERY_SERVER": "localhost:11811",
    "DNS_IP": "1.1.1.1",
    "GATEWAY_IP": public_gw_addr,
    "PASS": "turtlebot",
    "L2TP_LOCAL_IP": discovery_addr,
    "L2TP_REMOTE_IP": tunnel_dest_addr,
    "L2TP_ENDPOINT_CIDR": discovery_addr_l2tp + "/24",
}

discovery_labels = {
    "networks.1.interface": "eno12399np0",
    "networks.1.routes": route_towards_ue,
    "networks.1.ip": discovery_addr + "/24",
    "networks.2.interface": "ens1f1",
    "networks.2.ip": discovery_addr_public + "/27",
    "networks.2.gateway": public_gw_addr,
    "capabilities.privileged": "true",
}

try:
    chi.container.destroy_container(discovery_container_name)
    wait_until_container_removed(discovery_container_name)
    logger.success("Previous container destroyed.")
except:
    logger.info("No previous container found.")

discovery_container = chi.container.create_container(
    name = discovery_container_name,
    image = discovery_image_name,
    reservation_id = get_reservation_id_by_name(discovery_worker),
    environment = discovery_env_vars,
    nets = [
        { "network": edgenet['id'] },
        { "network": chi.network.get_network("serverpublic")['id'] },
    ],
    labels = discovery_labels,
)
chi.container.wait_for_active(discovery_container_name)
logger.success("Container deployed and active.")

# chi.container.execute(discovery_container_name, "ip link set eth0 down")

Start edge node ("host 2")

In [ ]:
edge_node_container_name = "listener-edge"
edge_node_image_name = "nilsjor/ros-humble-turtlebot:listener-v0.5"

edge_node_env_vars = {
    "ROS_DISCOVERY_SERVER": discovery_addr + ":11811",
}

edge_node_labels = {
    "networks.1.interface": "eno12409np1",
    "networks.1.routes": route_towards_ue,
    "networks.1.ip": edge_h2_addr + "/24",
    "capabilities.privileged": "true",
}

try:
    chi.container.destroy_container(edge_node_container_name)
    wait_until_container_removed(edge_node_container_name)
    logger.success("Previous container destroyed.")
except:
    logger.info("No previous container found.")

edge_node_container = chi.container.create_container(
    name = edge_node_container_name,
    image = edge_node_image_name,
    reservation_id = get_reservation_id_by_name(edge_node_worker),
    environment = edge_node_env_vars,
    nets = [
        { "network": edgenet['id'] },
    ],
    labels = edge_node_labels,
)

chi.container.wait_for_active(edge_node_container_name)
logger.success("Container deployed and active.")

# chi.container.execute(edge_node_container_name, "ip link set eth0 down")

Start UE1

In [ ]:
ue_node_container_name = "talker-device"
ue_node_image_name = "nilsjor/ros-humble-turtlebot:talker-v0.6"

ue_node_env_vars = {
    "ROS_DISCOVERY_SERVER": discovery_addr_l2tp + ":11811",
    "L2TP_LOCAL_IP": ue_priv_addr,
    "L2TP_REMOTE_IP": discovery_addr,
    "L2TP_ENDPOINT_CIDR": l2tp_h2_addr + "/24",
}
ue_node_labels = {
    "networks.1.interface": "eno12409",
    "networks.1.ip": ue_priv_addr + "/24",
    "networks.1.routes": route_towards_edge,
    "capabilities.privileged": "true",
}

try:
    chi.container.destroy_container(ue_node_container_name)
    wait_until_container_removed(ue_node_container_name)
    logger.success("Previous container destroyed.")
except:
    logger.info("No previous container found.")

ue_node_container = chi.container.create_container(
    name = ue_node_container_name,
    image = ue_node_image_name,
    reservation_id = get_reservation_id_by_name(ue1_node_worker),
    environment = ue_node_env_vars,
    nets = [
        { "network": chi.network.get_network("adv-04-net")['id'] },
    ],
    labels = ue_node_labels,
)

chi.container.wait_for_active(ue_node_container_name)
logger.success("Container deployed and active.")

# chi.container.execute(ue_node_container_name, "ip link set eth0 down")

## Teardown

Destroy all containers

In [ ]:
try:
    status = get_container_status(discovery_container_name)
    chi.container.destroy_container(discovery_container_name)
    wait_until_container_removed(discovery_container_name)
except:
    logger.info("No discovery container found.")

try:
    status = get_container_status(edge_node_container_name)
    chi.container.destroy_container(edge_node_container_name)
    wait_until_container_removed(edge_node_container_name)
except:
    logger.info("No talker container found.")

try:
    status = get_container_status(ue_node_container_name)
    chi.container.destroy_container(ue_node_container_name)
    wait_until_container_removed(ue_node_container_name)
except:
    logger.info("No listener container found.")

logger.info("Stopped and removed all containers")

Manually check if any "ghost" ports are left behind. Enter their ID (not name) in the next cell and run it to delete them.

In [ ]:
ghost_ports = [
    "ec7ef4c5-bc62-4af7-9f07-fd04bb0e2985",
]

for port in ghost_ports:
    try:
        chi.network.delete_port(port)
    except:
        logger.info(f"Port {port} not found.")

Proceed to clean up the rest of the project.

In [ ]:
# find the router again
router = None
try:
    router = chi.network.get_router("edge-router")
except Exception as ex:
    logger.info("Could not find edge-router.")

if router:
    # remove all routes from the router
    chi.network.remove_all_routes_from_router(router["id"])
    logger.success("Removed all routes from router.")

    # remove all subnets from the router
    subnets = chi.network.list_subnets()
    logger.info(f"Checking all {len(subnets)} subnets.")
    for subnet in subnets:
        try:
            chi.network.remove_subnet_from_router(router["id"],subnet["id"])
        except Exception as ex:
            pass
    logger.success("Removed all subnets from router")

    chi.network.delete_router(router["id"])
    logger.success("Deleted the router")

edgenet = None
try:
    edgenet = chi.network.get_network("edge-net")
except Exception as ex:
    logger.info("Could not find edge-net.")

if edgenet:
    chi.network.delete_network(edgenet["id"])
    logger.success("Deleted the edge-net")

Terminate reservations

In [ ]:
leaseslist = list_reservations(brief=True)
for lease in leaseslist:
    unreserve_byid(lease["id"])
    logger.success("Removed " + lease["name"])

logger.info("no leases remaining.")